In [34]:
import pandas as pd
import numpy as np
import re
import os, sys
import tiktoken
sys.path.append("..")
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/teamihajlov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/teamihajlov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# pd.set_option('display.max_colwidth', None)  # None means no limit on column width
# pd.set_option('display.width', 300)  

sci = pd.read_parquet("../datasets/science_parallel.parquet")
sci.head()

,title_en,title_sr,abstract_en,abstract_sr,kewywords_en,kewywords_sr
0,The origins and development of the architectur...,Poreklo i razvoj arhitektonske forme rimokatol...,The aim of the paper is to present an architec...,Predmet rada je proučavanje arhitekture rimoka...,"[Sacral architecture, roman catholic church, a...","[Sakralna arhitektura, rimokatlička Crkva, arh..."
1,Belgrade's General Plan 1923: a comparison of ...,Generalni plan Beograda 1923 komparacija plani...,The doctoral dissertation examines the process...,U doktorskoj disertaciji je istraživan proces ...,"[History, theory, urban lanning, general plan,...","[istorija, teorija, urbanističko planiranje, g..."
2,The importance and role of pedestrian space ne...,Značaj i uloga mreže pešačkih prostora u gener...,City represents its character through urban/pl...,Svoj karakter grad predstavlja putem urbanog m...,"[Pedestrian space network, communication of th...","[Mreža pešačkih prostora, komunikacija grada, ..."
3,Adaptive principles in architectural design,Adaptivni principi u arhitektonskom projektovanju,This study is aiming at the development of arc...,Ovaj rad predstavlja prilog razvoju metodologi...,"[adaptation, analogue model, performative mode...","[adaptacija, analogni model, perfomativni mode..."
4,Transformation of vernacular architecture on t...,Transformacija narodne arhitekture na poluostr...,Vrmac Peninsula is located in the Bay of Kotor...,Poluostrvo Vrmac nalazi se u Bokokotorskom zal...,"[peninsula Vrmac, vernacular transformation, c...","[poluostrvo Vrmac, transformacija narodnih kuć..."


In [4]:
sci['abstract_en'] = sci['abstract_en'].replace('', np.nan)
sci['abstract_sr'] = sci['abstract_sr'].replace('', np.nan)
sci_cleaned = sci.dropna(subset=['abstract_en', 'abstract_sr'], ignore_index=True)
sci_cleaned.shape

(9962, 6)

In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """
    Calculates the number of tokens in a string based on a specified encoding.

    The function uses the specified encoding to tokenize the input string and returns the number of tokens generated.

    Parameters:
    string (str): The input string to be tokenized.
    encoding_name (str): The name of the encoding to use for tokenization.

    Returns:
    int: The number of tokens in the input string based on the specified encoding.

    Example:
    >>> string = "This is a sample sentence."
    >>> encoding_name = "cl100k_base"
    >>> num_tokens = num_tokens_from_string(string, encoding_name)
    >>> print(num_tokens)
    """
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [6]:
tokens_sr_processed = [num_tokens_from_string(i, "cl100k_base") for i in sci_cleaned['abstract_sr']]

In [7]:
print(f"Mean (tokens): {np.mean(tokens_sr_processed)}")
print(f"Max length (tokens): {np.max(tokens_sr_processed)}")
print(f"Min length (tokens): {np.min(tokens_sr_processed)}")


Mean (tokens): 880.2607910058222
Max length (tokens): 11258
Min length (tokens): 16


In [8]:
def tokenize_and_truncate(text, encoding_name="cl100k_base", max_tokens=512):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    num_tokens = len(tokens)
    if num_tokens > max_tokens:
        tokens = tokens[:max_tokens]  # Truncate to 512 tokens
    processed_abstract = encoding.decode(tokens)  # Decode back to text
    if not processed_abstract.endswith('.'):
        last_period_index = processed_abstract.rfind('.')
        if last_period_index != -1:
            processed_abstract = processed_abstract[:last_period_index + 1]  # Keep everything up to the last "."
            
    return processed_abstract


In [9]:
sci_cleaned['processed_abstract_sr'] = sci_cleaned["abstract_sr"].apply(tokenize_and_truncate)
sci_cleaned['processed_abstract_en'] = sci_cleaned["abstract_en"].apply(tokenize_and_truncate)

/var/folders/dt/l5k05lj153n36h3p7st40sy00000gn/T/ipykernel_16645/2407012175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sci_cleaned['processed_abstract_sr'] = sci_cleaned["abstract_sr"].apply(tokenize_and_truncate)
/var/folders/dt/l5k05lj153n36h3p7st40sy00000gn/T/ipykernel_16645/2407012175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sci_cleaned['processed_abstract_en'] = sci_cleaned["abstract_en"].apply(tokenize_and_truncate)


In [10]:
sci_cleaned.head()

,title_en,title_sr,abstract_en,abstract_sr,kewywords_en,kewywords_sr,processed_abstract_sr,processed_abstract_en
0,The origins and development of the architectur...,Poreklo i razvoj arhitektonske forme rimokatol...,The aim of the paper is to present an architec...,Predmet rada je proučavanje arhitekture rimoka...,"[Sacral architecture, roman catholic church, a...","[Sakralna arhitektura, rimokatlička Crkva, arh...",Predmet rada je proučavanje arhitekture rimoka...,The aim of the paper is to present an architec...
1,Belgrade's General Plan 1923: a comparison of ...,Generalni plan Beograda 1923 komparacija plani...,The doctoral dissertation examines the process...,U doktorskoj disertaciji je istraživan proces ...,"[History, theory, urban lanning, general plan,...","[istorija, teorija, urbanističko planiranje, g...",U doktorskoj disertaciji je istraživan proces ...,The doctoral dissertation examines the process...
2,The importance and role of pedestrian space ne...,Značaj i uloga mreže pešačkih prostora u gener...,City represents its character through urban/pl...,Svoj karakter grad predstavlja putem urbanog m...,"[Pedestrian space network, communication of th...","[Mreža pešačkih prostora, komunikacija grada, ...",Svoj karakter grad predstavlja putem urbanog m...,City represents its character through urban/pl...
3,Adaptive principles in architectural design,Adaptivni principi u arhitektonskom projektovanju,This study is aiming at the development of arc...,Ovaj rad predstavlja prilog razvoju metodologi...,"[adaptation, analogue model, performative mode...","[adaptacija, analogni model, perfomativni mode...",Ovaj rad predstavlja prilog razvoju metodologi...,This study is aiming at the development of arc...
4,Transformation of vernacular architecture on t...,Transformacija narodne arhitekture na poluostr...,Vrmac Peninsula is located in the Bay of Kotor...,Poluostrvo Vrmac nalazi se u Bokokotorskom zal...,"[peninsula Vrmac, vernacular transformation, c...","[poluostrvo Vrmac, transformacija narodnih kuć...",Poluostrvo Vrmac nalazi se u Bokokotorskom zal...,Vrmac Peninsula is located in the Bay of Kotor...


In [11]:
tokens_sr_processed = [num_tokens_from_string(i, "cl100k_base") for i in sci_cleaned['processed_abstract_sr']]

idxs_sr = []
for idx, i in enumerate(tokens_sr_processed):
    if i == 0:
        idxs_sr.append(idx)

print(f"Mean (tokens): {np.mean(tokens_sr_processed)}")
print(f"Max length (tokens): {np.max(tokens_sr_processed)}")
print(f"Min length (tokens): {np.min(tokens_sr_processed)}")
print(f"Missing: {len(idxs_sr)}")

Mean (tokens): 431.1939369604497
Max length (tokens): 512
Min length (tokens): 16
Missing: 0


In [12]:
tokens_en = [num_tokens_from_string(i, "cl100k_base") for i in sci_cleaned['processed_abstract_en']]

idxs_en = []
for idx_en, i in enumerate(tokens_en):
    if i == 0:
        idxs_en.append(idx)

print(f"Mean (tokens): {np.mean(tokens_en)}")
print(f"Max length (tokens): {np.max(tokens_en)}")
print(f"Min length (tokens): {np.min(tokens_en)}")
print(f"Missing: {len(idxs_en)}")

Mean (tokens): 370.8576591045975
Max length (tokens): 512
Min length (tokens): 8
Missing: 0


In [13]:
def extract_first_sentence(abstract):
    return abstract.split('.')[0] + '.' if '.' in abstract else abstract

In [14]:
df_sentences = pd.DataFrame({
    'sentence_en': sci_cleaned['processed_abstract_en'].apply(extract_first_sentence),
    'sentence_sr': sci_cleaned['processed_abstract_sr'].apply(extract_first_sentence)
})

In [15]:
df_sentences.head()

,sentence_en,sentence_sr
0,The aim of the paper is to present an architec...,Predmet rada je proučavanje arhitekture rimoka...
1,The doctoral dissertation examines the process...,U doktorskoj disertaciji je istraživan proces ...
2,City represents its character through urban/pl...,Svoj karakter grad predstavlja putem urbanog m...
3,This study is aiming at the development of arc...,Ovaj rad predstavlja prilog razvoju metodologi...
4,Vrmac Peninsula is located in the Bay of Kotor.,Poluostrvo Vrmac nalazi se u Bokokotorskom zal...


In [56]:
df_title = pd.DataFrame({"sentence_en": sci['title_en'],
                         "sentence_sr": sci['title_sr']})
df_title

,sentence_en,sentence_sr
0,The origins and development of the architectur...,Poreklo i razvoj arhitektonske forme rimokatol...
1,Belgrade's General Plan 1923: a comparison of ...,Generalni plan Beograda 1923 komparacija plani...
2,The importance and role of pedestrian space ne...,Značaj i uloga mreže pešačkih prostora u gener...
3,Adaptive principles in architectural design,Adaptivni principi u arhitektonskom projektovanju
4,Transformation of vernacular architecture on t...,Transformacija narodne arhitekture na poluostr...
...,...,...
13284,An interesting relationship between finite rin...,Konačni prsteni i usmereni grafovi
13285,Ekstremne vrednosti u nizovima nezavisnih sluč...,Extreme values in sequences of independent ran...
13286,An improved method for the determination of th...,Unapređena metoda za određivanje refleksionih ...
13287,Hub maximal covering problems - novel mathemat...,Problemi hab maksimalnog pokrivanja - novi mat...


In [35]:
sentences_sr = [sent_tokenize(text) for text in sci_cleaned['processed_abstract_sr']]
print(sentences_sr[0])
print(len(sentences_sr))

['Predmet rada je proučavanje arhitekture rimokatoličkih crkava građenih u periodu od 1699. do 1939. godine na području Vojvodine.', 'U vremenima koja su usledila po završetku austro-turskih ratova i konačnog prestanka dominacije Otomanske imperije na prostoru Habsburške monarhije (1745- 1918, odnosno carevine Austrije od 1804. i Austro-Ugarske monarhističke unije od 1867), sa potpisivanjem Karlovačkog (1699) i Požarevačkog mira (1718), stvoreni su uslovi za obnovu južnih Ugarskih oblasti.', 'Vreme društvenog i ekonomskog prosperiteta, koji je trajao sve do izbijanja Prvog svetskog rata, stvorilo je uslove za sveobuhvatnu ekspanziju graditeljskih aktivnosti.', 'Kontinuitet izgradnje sakralnih građevina državno dominantne rimokatoličke konfesije, može se pratiti i nakon ovog perioda, sve do izbijanja Drugog svetskog rata (1939.).', 'Tokom skoro dva i po veka izgrađen je na području Vojvodine 231 rimokatolički sakralni objekat, od čega 3 katedralna hrama, 184 župne crkve i 44 zavetne, po

In [57]:
df_concaten = pd.concat([df_sentences, df_title], ignore_index=True)
df_concaten

,sentence_en,sentence_sr
0,The aim of the paper is to present an architec...,Predmet rada je proučavanje arhitekture rimoka...
1,The doctoral dissertation examines the process...,U doktorskoj disertaciji je istraživan proces ...
2,City represents its character through urban/pl...,Svoj karakter grad predstavlja putem urbanog m...
3,This study is aiming at the development of arc...,Ovaj rad predstavlja prilog razvoju metodologi...
4,Vrmac Peninsula is located in the Bay of Kotor.,Poluostrvo Vrmac nalazi se u Bokokotorskom zal...
...,...,...
23246,An interesting relationship between finite rin...,Konačni prsteni i usmereni grafovi
23247,Ekstremne vrednosti u nizovima nezavisnih sluč...,Extreme values in sequences of independent ran...
23248,An improved method for the determination of th...,Unapređena metoda za određivanje refleksionih ...
23249,Hub maximal covering problems - novel mathemat...,Problemi hab maksimalnog pokrivanja - novi mat...


In [59]:
df_concaten.isna().sum()

sentence_en    0
sentence_sr    0
dtype: int64

In [58]:
df_concaten.to_parquet("../datasets/sci_parallel_abstract_sentence_and_title.parquet")

In [43]:
lenghts_sr = [len(i) for i in sentences_sr]
print(lenghts_sr)

[5, 1, 7, 1, 6, 4, 5, 5, 9, 6, 7, 4, 6, 7, 6, 3, 7, 5, 11, 6, 4, 1, 8, 5, 1, 5, 5, 4, 1, 5, 6, 3, 8, 5, 5, 7, 5, 3, 6, 12, 8, 7, 5, 6, 4, 8, 2, 1, 4, 6, 4, 7, 2, 6, 2, 6, 1, 1, 9, 8, 6, 7, 7, 7, 6, 4, 7, 5, 3, 6, 7, 5, 7, 7, 1, 5, 7, 4, 5, 11, 4, 5, 8, 1, 1, 6, 6, 7, 3, 6, 4, 2, 4, 3, 6, 11, 9, 4, 5, 5, 1, 5, 8, 9, 9, 3, 8, 7, 7, 10, 6, 5, 6, 6, 9, 8, 6, 3, 6, 1, 10, 5, 4, 6, 6, 7, 7, 6, 9, 9, 5, 6, 9, 11, 6, 7, 7, 6, 5, 7, 9, 5, 6, 6, 7, 7, 9, 6, 1, 3, 10, 7, 7, 8, 1, 10, 11, 5, 6, 5, 7, 7, 5, 8, 6, 11, 8, 4, 6, 4, 1, 5, 11, 6, 6, 5, 5, 8, 6, 10, 9, 1, 7, 7, 7, 11, 9, 8, 7, 5, 8, 8, 8, 7, 12, 7, 8, 11, 9, 6, 7, 6, 6, 5, 4, 6, 5, 11, 6, 10, 7, 8, 10, 8, 7, 8, 11, 5, 8, 7, 10, 11, 7, 9, 6, 8, 10, 3, 8, 7, 9, 6, 9, 8, 7, 6, 8, 11, 3, 15, 6, 9, 8, 6, 9, 8, 9, 8, 8, 12, 5, 7, 8, 8, 7, 7, 5, 9, 6, 7, 10, 6, 8, 9, 8, 8, 6, 10, 9, 10, 10, 8, 7, 8, 7, 6, 5, 10, 10, 8, 5, 7, 6, 6, 5, 10, 9, 6, 9, 12, 5, 9, 7, 10, 7, 8, 10, 10, 8, 11, 10, 10, 7, 6, 9, 6, 8, 9, 6, 6, 7, 8, 8, 7, 8, 8, 5, 8, 5, 6,

In [36]:
sentences_en = [sent_tokenize(text) for text in sci_cleaned['processed_abstract_en']]
print(sentences_en[0])
print(len(sentences_en))

['The aim of the paper is to present an architectonic study of Roman Catholic churches built in Vojvodina in years 1699. to 1939.', 'The revitalization of the southern Hungarian region followed the end of the domination of the Ottoman Empire in the lands of the Habsburg Monarchy (1745-1918, Austrian Empire, 1804-1867.', '& Austro-Hungarian Empire, 1867-1918.)', 'in the times after the Austro-Turkish wars and the signing of the Karlovci (1699.)', 'and Pozarevac (1718.)', 'Peace Treats.', 'The period of political and economic stability, which lasted until the outbreak of the First World War, was favorable time for a comprehensive expansion of constructing activities.', 'The continuity of the construction of religious buildings of the state-dominant Roman Catholic religion, can be followed after this period, until the outbreak of the Second World War (1939).', 'For nearly two and a half centuries on the territory of Vojvodina, 231 Roman Catholic religious buildings were built, including 3

In [44]:
lenghts_en = [len(i) for i in sentences_en]
print(lenghts_en)

[16, 1, 9, 1, 10, 7, 6, 10, 9, 8, 11, 5, 11, 14, 11, 4, 13, 7, 10, 14, 2, 1, 9, 11, 1, 7, 6, 8, 1, 12, 11, 1, 16, 4, 10, 8, 7, 7, 10, 11, 15, 9, 9, 11, 14, 21, 2, 1, 6, 9, 4, 10, 2, 12, 1, 13, 1, 1, 15, 8, 5, 15, 10, 10, 6, 6, 19, 14, 3, 10, 10, 7, 15, 11, 1, 5, 9, 8, 13, 17, 4, 5, 12, 2, 2, 6, 8, 7, 2, 10, 1, 2, 6, 3, 6, 11, 13, 4, 7, 6, 1, 9, 8, 8, 15, 4, 18, 9, 10, 16, 10, 8, 11, 10, 14, 12, 12, 9, 12, 2, 12, 9, 7, 11, 9, 10, 15, 14, 10, 13, 13, 9, 11, 13, 12, 12, 15, 11, 12, 8, 15, 10, 9, 8, 11, 9, 14, 9, 1, 3, 14, 10, 13, 10, 1, 11, 15, 8, 11, 12, 13, 17, 7, 12, 12, 12, 11, 9, 9, 11, 1, 7, 13, 12, 11, 10, 6, 8, 9, 13, 17, 1, 8, 14, 14, 14, 15, 10, 11, 5, 12, 11, 11, 10, 16, 8, 14, 15, 13, 14, 9, 13, 12, 10, 6, 10, 10, 15, 10, 12, 9, 15, 15, 13, 8, 15, 19, 10, 12, 7, 18, 16, 11, 15, 11, 14, 16, 9, 9, 12, 17, 11, 15, 14, 10, 13, 11, 17, 3, 23, 12, 13, 11, 17, 10, 11, 10, 15, 14, 17, 9, 12, 12, 11, 12, 10, 12, 17, 8, 12, 18, 11, 9, 12, 12, 11, 16, 10, 11, 15, 11, 11, 10, 11, 12, 10, 

In [46]:
def count_differences(list1, list2):
    # Ensure both lists have the same length
    if len(list1) != len(list2):
        raise ValueError("Lists must be of the same length.")

    difference_count = 0
    for a, b in zip(list1, list2):
        if a != b:
            difference_count += 1
            
    return difference_count

# Example usage
list1 = [1, 2, 3, 4, 5]
list2 = [1, 3, 2, 4, 6]
diff_count = count_differences(lenghts_sr, lenghts_en)
print(f"Number of differing values: {diff_count}")

Number of differing values: 8553


In [40]:
flattened_list = [string for sublist in sentences_sr for string in sublist]

# Create a DataFrame
df_sr = pd.DataFrame(flattened_list, columns=["sentences_sr"])

# Display the DataFrame
print(df_sr)

                                            sentences_sr
0      Predmet rada je proučavanje arhitekture rimoka...
1      U vremenima koja su usledila po završetku aust...
2      Vreme društvenog i ekonomskog prosperiteta, ko...
3      Kontinuitet izgradnje sakralnih građevina drža...
4      Tokom skoro dva i po veka izgrađen je na podru...
...                                                  ...
66890  Brizantno dejstvo eksplozije izraženo je dejst...
66891  Od policije i sudskih veštaka sud traži određi...
66892  Očekivani doprinos ovog rada je u oblasti fore...
66893  U prilog opravdanosti istraživanja data je ana...
66894                    na teritoriji Republike Srbije.

[66895 rows x 1 columns]


In [47]:
df_sr.to_parquet("../datasets/science_sentences_sr.parquet")

In [39]:
flattened_list = [string for sublist in sentences_en for string in sublist]

# Create a DataFrame
df_en = pd.DataFrame(flattened_list, columns=["sentences_en"])
# Display the DataFrame
print(df_en)

                                             sentences_en
0       The aim of the paper is to present an architec...
1       The revitalization of the southern Hungarian r...
2                  & Austro-Hungarian Empire, 1867-1918.)
3       in the times after the Austro-Turkish wars and...
4                                   and Pozarevac (1718.)
...                                                   ...
104816  Activation of explosive devices, whether inten...
104817  From the forensic aspect the damage in the for...
104818  Dimensions and shape of the crater are closely...
104819  Calculation of the mass of the explosive used ...
104820  This paper contains the review of the existing...

[104821 rows x 1 columns]


In [48]:
df_en.to_parquet("../datasets/science_sentences_en.parquet")